In [60]:
from tqdm import tqdm
import pandas as pd
from binance.client import Client

import numpy as np
import ta
from ta.volatility import BollingerBands, AverageTrueRange
import re as re
from sqlalchemy.sql import exists  
from sqlalchemy import select
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker, relationship

In [61]:
client = Client()

In [62]:
from sqlalchemy import create_engine

pd.set_option("display.max_rows", 100, "display.max_columns", 100)

In [63]:
coins = ('BTCUSDT','ETHUSDT','BNBUSDT','SOLUSDT','ADAUSDT','XRPUSDT','DOTUSDT','LUNAUSDT',
  'DOGEUSDT','AVAXUSDT','SHIBUSDT','MATICUSDT','LTCUSDT','UNIUSDT','ALGOUSDT','TRXUSDT',
         'LINKUSDT','MANAUSDT','ATOMUSDT','VETUSDT')

In [64]:
fast = 7
slow = 25


In [65]:
def getminutedata(symbol, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'1m',lookback + ' days ago UTC' ))

    frame = frame.iloc[:,:6]
    frame.drop_duplicates()
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame[['Open','High','Low','Close','Volume']] = frame[['Open','High','Low','Close','Volume']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit='ms')
    return frame

In [66]:
engine = db.create_engine('sqlite:///Cryptoprices3.db', echo = True)

In [67]:
Session = sessionmaker(bind=engine)
session = Session()

In [68]:
#users = session.query(Time)
#exists = db.session.query(User.id).filter_by(name='davidism').first() is not None

In [69]:
#

In [70]:
for coin in tqdm(coins):
    getminutedata(coin,'1').to_sql(coin, engine,if_exists="append", index=False)


  0%|          | 0/20 [00:00<?, ?it/s]

2022-01-16 22:55:34,931 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("BTCUSDT")
2022-01-16 22:55:34,932 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:55:34,935 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("BTCUSDT")
2022-01-16 22:55:34,936 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:55:34,941 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-16 22:55:34,943 INFO sqlalchemy.engine.Engine 
CREATE TABLE "BTCUSDT" (
	"Time" DATETIME, 
	"Open" FLOAT, 
	"High" FLOAT, 
	"Low" FLOAT, 
	"Close" FLOAT, 
	"Volume" FLOAT
)


2022-01-16 22:55:34,945 INFO sqlalchemy.engine.Engine [no key 0.00199s] ()
2022-01-16 22:55:34,957 INFO sqlalchemy.engine.Engine COMMIT
2022-01-16 22:55:34,975 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-16 22:55:34,998 INFO sqlalchemy.engine.Engine INSERT INTO "BTCUSDT" ("Time", "Open", "High", "Low", "Close", "Volume") VALUES (?, ?, ?, ?, ?, ?)
2022-01-16 22:55:34,999 INFO sqlalchemy.engine.Engine [generated in 0.01905s

  5%|▌         | 1/20 [00:01<00:26,  1.40s/it]

2022-01-16 22:55:37,110 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ETHUSDT")
2022-01-16 22:55:37,112 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:55:37,114 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ETHUSDT")
2022-01-16 22:55:37,116 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:55:37,120 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-16 22:55:37,122 INFO sqlalchemy.engine.Engine 
CREATE TABLE "ETHUSDT" (
	"Time" DATETIME, 
	"Open" FLOAT, 
	"High" FLOAT, 
	"Low" FLOAT, 
	"Close" FLOAT, 
	"Volume" FLOAT
)


2022-01-16 22:55:37,124 INFO sqlalchemy.engine.Engine [no key 0.00170s] ()
2022-01-16 22:55:37,134 INFO sqlalchemy.engine.Engine COMMIT
2022-01-16 22:55:37,154 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-16 22:55:37,175 INFO sqlalchemy.engine.Engine INSERT INTO "ETHUSDT" ("Time", "Open", "High", "Low", "Close", "Volume") VALUES (?, ?, ?, ?, ?, ?)
2022-01-16 22:55:37,176 INFO sqlalchemy.engine.Engine [generated in 0.01702s

 10%|█         | 2/20 [00:03<00:33,  1.86s/it]

...

 90%|█████████ | 18/20 [00:33<00:02,  1.35s/it]

2022-01-16 22:56:08,238 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ATOMUSDT")
2022-01-16 22:56:08,240 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:08,243 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ATOMUSDT")
2022-01-16 22:56:08,246 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:08,253 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-16 22:56:08,256 INFO sqlalchemy.engine.Engine 
CREATE TABLE "ATOMUSDT" (
	"Time" DATETIME, 
	"Open" FLOAT, 
	"High" FLOAT, 
	"Low" FLOAT, 
	"Close" FLOAT, 
	"Volume" FLOAT
)


2022-01-16 22:56:08,258 INFO sqlalchemy.engine.Engine [no key 0.00204s] ()
2022-01-16 22:56:08,271 INFO sqlalchemy.engine.Engine COMMIT
2022-01-16 22:56:08,293 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-16 22:56:08,334 INFO sqlalchemy.engine.Engine INSERT INTO "ATOMUSDT" ("Time", "Open", "High", "Low", "Close", "Volume") VALUES (?, ?, ?, ?, ?, ?)
2022-01-16 22:56:08,335 INFO sqlalchemy.engine.Engine [generated in 0.03

 95%|█████████▌| 19/20 [00:34<00:01,  1.34s/it]

2022-01-16 22:56:09,672 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("VETUSDT")
2022-01-16 22:56:09,674 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:09,679 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("VETUSDT")
2022-01-16 22:56:09,680 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:09,686 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-16 22:56:09,688 INFO sqlalchemy.engine.Engine 
CREATE TABLE "VETUSDT" (
	"Time" DATETIME, 
	"Open" FLOAT, 
	"High" FLOAT, 
	"Low" FLOAT, 
	"Close" FLOAT, 
	"Volume" FLOAT
)


2022-01-16 22:56:09,691 INFO sqlalchemy.engine.Engine [no key 0.00322s] ()
2022-01-16 22:56:09,703 INFO sqlalchemy.engine.Engine COMMIT
2022-01-16 22:56:09,719 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-16 22:56:09,740 INFO sqlalchemy.engine.Engine INSERT INTO "VETUSDT" ("Time", "Open", "High", "Low", "Close", "Volume") VALUES (?, ?, ?, ?, ?, ?)
2022-01-16 22:56:09,742 INFO sqlalchemy.engine.Engine [generated in 0.01897s

100%|██████████| 20/20 [00:36<00:00,  1.81s/it]


In [71]:
df = test=pd.read_sql('BTCUSDT',engine).set_index('Time')

2022-01-16 22:56:09,824 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("BTCUSDT")
2022-01-16 22:56:09,825 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:09,829 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-01-16 22:56:09,830 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:09,834 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("BTCUSDT")
2022-01-16 22:56:09,836 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:09,839 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-01-16 22:56:09,840 INFO sqlalchemy.engine.Engine [raw sql] ('BTCUSDT',)
2022-01-16 22:56:09,843 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("BTCUSDT")
2022-01-16 22:56:09,845 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:09,846 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("BT

In [72]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-01-15 21:56:00,43289.63,43331.42,43288.55,43326.23,30.58938
2022-01-15 21:57:00,43324.50,43335.42,43310.06,43311.23,9.38515
2022-01-15 21:58:00,43313.58,43330.32,43309.17,43318.37,4.68691
2022-01-15 21:59:00,43318.37,43327.33,43294.20,43296.72,14.56433
2022-01-15 22:00:00,43296.72,43335.00,43296.71,43330.59,10.65186
...,...,...,...,...,...
2022-01-16 21:51:00,42973.51,42999.99,42965.18,42971.53,7.10585
2022-01-16 21:52:00,42968.34,42999.42,42968.34,42990.88,7.52637
2022-01-16 21:53:00,42990.88,43004.77,42988.44,43000.01,4.19960


In [73]:
def groups_from_conditions(cond1, cond2):
    '''
    assign a unique non-NaN integer to each group as defined by the rules
    '''
    n = len(cond1)
    
    group_idx = -1
    groups = np.zeros(n)

    curr_state = 0    # 0 = not in a group, 1 = in a group
    for n in range(n):
        if curr_state == 0:
            # Currently not in a group
            if cond1[n] == 1:
                # Detected start of a group. so:
                # switch the state to 1 ie in a group
                curr_state = 1
                # get a new group_idx
                group_idx = group_idx + 1
                # assign it to the output for element n
                groups[n] = group_idx
            else:
                # no start of the group detected, we are not in a group so mark as NaN
                groups[n] = np.NaN

        else: 
            # current_state == 1 so we are in a group
            if cond2[n] == 1:
                # detected end of group -- switch state to 0
                curr_state = 0
            # as we are in a group assign current group_idx. Note that this happens for the element
            # for which cond2[n] == 1 as well, ie this element is included
            groups[n] = group_idx

    return groups

In [74]:

def getMax(df2):
    test_cond1 = df2['buy'].values
    test_cond2 = df2['sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    test_df = pd.DataFrame({'buy' : test_cond1, 'sell' : test_cond2, 'groups' : test_groups})
    max = test_df['groups'].max()
    return max

In [75]:
class Strat:
  def __init__(self, name, age):
    self.name = name
    self.age = age

  def myfunc(self):
    print("Hello my name is " + self.name)

In [76]:


def technicals(df):
    df = df.copy()
    df = df.resample('5min').ffill()
    df.dropna(inplace=True)
    df['return'] = np.log(df.Close.pct_change()+ 1)
    df['SMA_fast'] = df.Close.rolling(7).mean()
    df['SMA_slow'] = df.Close.rolling(25).mean()
    df['position'] = np.where(df['SMA_fast'] > df['SMA_slow'],1,0)  # true or false = 1 eller 0
    df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
    df.dropna(inplace=True)
    return df


In [77]:
def bb(df):
    df = df.copy()
    df = df.resample('5min').ffill()
    df.dropna(inplace=True)
    bb_indicator = BollingerBands(df['Close'])
    df['return'] = np.log(df.Close.pct_change()+ 1)
    df['bb_high'] = bb_indicator.bollinger_hband()
    df['bb_low'] = bb_indicator.bollinger_lband()
    df['moving_avg'] = bb_indicator.bollinger_mavg()
    df['buy'] = np.where(df['Close'] < df['bb_low'], 1, 0).astype(float)
    df['sell'] = np.where(df['Close'] > df['bb_high'], 1, 0).astype(float)
    test_cond1 = df['buy'].values
    test_cond2 = df['sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    df['groups'] = test_groups
    df['position'] = np.where(df['groups'].isnull(),0,1)
    df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
    df.to_string("x")
    df.dropna(inplace=True)
    return df

In [78]:
for coin in coins:
    df = pd.read_sql(coin, engine).set_index('Time')
    print(coin)
    trades = technicals(df).position.diff().value_counts().iloc[1:].sum()
    print(trades)
    costs = trades * 0.00075
    print(np.exp(technicals(df)['return'].sum())-1)
    print(np.exp(technicals(df)[['strategyreturn']].sum())-1-costs)

    #iloc[1: exclude the first value which is total number of trades]

2022-01-16 22:56:10,483 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("BTCUSDT")
2022-01-16 22:56:10,484 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:10,490 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-01-16 22:56:10,492 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:10,497 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("BTCUSDT")
2022-01-16 22:56:10,499 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:10,504 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-01-16 22:56:10,505 INFO sqlalchemy.engine.Engine [raw sql] ('BTCUSDT',)
2022-01-16 22:56:10,510 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("BTCUSDT")
2022-01-16 22:56:10,511 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:10,514 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("BT

In [79]:
for coin in coins:
    df = pd.read_sql(coin, engine).set_index('Time')
    print(coin)
    trades = bb(df)['groups'].max()
    print(trades)
    costs = trades * 0.00075
    print(np.exp(bb(df)['return'].sum())-1)
    print(np.exp(bb(df)[['return','strategyreturn']].sum())-1-costs)


    #iloc[1: exclude the first value which is total number of trades]


    

2022-01-16 22:56:14,695 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("BTCUSDT")
2022-01-16 22:56:14,697 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:14,703 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-01-16 22:56:14,705 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:14,709 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("BTCUSDT")
2022-01-16 22:56:14,710 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:14,716 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-01-16 22:56:14,717 INFO sqlalchemy.engine.Engine [raw sql] ('BTCUSDT',)
2022-01-16 22:56:14,721 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("BTCUSDT")
2022-01-16 22:56:14,723 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-16 22:56:14,725 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("BT

In [80]:
bb(df).to_string('sdsdsssssss')  


In [81]:
##df['EMA200'] = ta.trend.ema_indicator(df.Close, window=200)
#df['wf_Top_bool'] = np.where(df['High'].rolling(9, center=True).max(), True, False)

#0.075/100

In [82]:
bb(df)

,Open,High,Low,Close,Volume,return,bb_high,bb_low,moving_avg,buy,sell,groups,position,strategyreturn
Time,,,,,,,,,,,,,,
2022-01-16 00:05:00,0.07841,0.07841,0.07819,0.07823,691961.9,-0.003828,0.079171,0.078327,0.078749,1.0,0.0,0.0,1,-0.000000
2022-01-16 00:10:00,0.07837,0.07852,0.07837,0.07851,376276.7,0.003573,0.079166,0.078301,0.078733,0.0,0.0,0.0,1,0.003573
2022-01-16 00:15:00,0.07839,0.07843,0.07830,0.07835,1689491.6,-0.002040,0.079180,0.078252,0.078716,0.0,0.0,0.0,1,-0.002040
2022-01-16 00:20:00,0.07853,0.07855,0.07842,0.07842,189037.6,0.000893,0.079185,0.078223,0.078704,0.0,0.0,0.0,1,0.000893
2022-01-16 00:25:00,0.07851,0.07851,0.07849,0.07850,12297.6,0.001020,0.079183,0.078205,0.078694,0.0,0.0,0.0,1,0.001020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-16 21:35:00,0.07802,0.07807,0.07799,0.07806,137955.4,-0.002303,0.078555,0.078081,0.078318,1.0,0.0,4.0,1,-0.002303
2022-01-16 21:40:00,0.07797,0.07797,0.07793,0.07793,78267.7,-0.001667,0.078582,0.078007,0.078295,1.0,0.0,4.0,1,-0.001667
2022-01-16 21:45:00,0.07801,0.07807,0.07801,0.07803,373185.3,0.001282,0.078593,0.077984,0.078288,0.0,0.0,4.0,1,0.001282


In [83]:
def profits(df2):
    getMax(df2)
    fakeprofit = df2.groupby(groups_from_conditions(df2['buy'].values, df2['sell'].values))['return'].sum().to_frame().sum()
    fee = getMax(df2)*0.00075
    profit = fakeprofit - fee
    return profit